In [1]:
import sys
print(sys.executable)


# Manejo de datos
import pandas as pd
import numpy as np

# Visualización (opcional, pero útil)
import matplotlib.pyplot as plt
import seaborn as sns

# Optimización (iremos usando una u otra)
from ortools.sat.python import cp_model   # muy potente para horarios
# import pulp                              # alternativa lineal
# import pyomo.environ as pyo              # alternativa académica

# Utilidades
from itertools import product


c:\Users\MEDIA\anaconda3\envs\horarios310\python.exe


In [2]:
RUTA_CSV = "data/prueba_horario_instituto.csv"

df = pd.read_csv(RUTA_CSV)

print("CSV cargado correctamente")
print(f"Número de profesores: {df['id_profesor'].nunique()}")
print(f"Número de columnas: {df.shape[1]}")

CSV cargado correctamente
Número de profesores: 110
Número de columnas: 40


In [3]:
# Vista de las primeras filas
df.head()


,id_profesor,1BACH_A,1BACH_B,1BACH_C,1BACH_D,1BACH_E,1BACH_F,1E_A,1E_B,1E_C,...,3E_D,3E_E,3E_F,4E_A,4E_B,4E_C,4E_D,4E_E,4E_F,restricciones
0,BIO1,0,0,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,1_2
1,BIO2,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,NaN
2,BIO3,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,NaN
3,BIO4,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,5_5
4,BIO5,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,NaN


In [4]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 40 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id_profesor    110 non-null    object
 1   1BACH_A        110 non-null    int64 
 2   1BACH_B        110 non-null    int64 
 3   1BACH_C        110 non-null    int64 
 4   1BACH_D        110 non-null    int64 
 5   1BACH_E        110 non-null    int64 
 6   1BACH_F        110 non-null    int64 
 7   1E_A           110 non-null    int64 
 8   1E_B           110 non-null    int64 
 9   1E_C           110 non-null    int64 
 10  1E_D           110 non-null    int64 
 11  1E_E           110 non-null    int64 
 12  1E_F           110 non-null    int64 
 13  1E_G           110 non-null    int64 
 14  2BACH_A        110 non-null    int64 
 15  2BACH_B        110 non-null    int64 
 16  2BACH_C        110 non-null    int64 
 17  2BACH_D        110 non-null    int64 
 18  2BACH_E        110 non-null   

In [5]:
df.describe()


,1BACH_A,1BACH_B,1BACH_C,1BACH_D,1BACH_E,1BACH_F,1E_A,1E_B,1E_C,1E_D,...,3E_C,3E_D,3E_E,3E_F,4E_A,4E_B,4E_C,4E_D,4E_E,4E_F
count,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,...,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000
mean,0.100000,0.100000,0.100000,0.100000,0.100000,0.090909,0.100000,0.100000,0.100000,0.100000,...,0.118182,0.118182,0.118182,0.118182,0.109091,0.109091,0.109091,0.109091,0.109091,0.109091
std,0.301373,0.301373,0.301373,0.301373,0.301373,0.288795,0.301373,0.301373,0.301373,0.301373,...,0.324301,0.324301,0.324301,0.324301,0.313180,0.313180,0.313180,0.313180,0.313180,0.313180
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
col_restr = "restricciones"  # ajusta si el nombre exacto es otro

profes_con_restr = df[df[col_restr].notna() & (df[col_restr] != "")]

print(f"Número de profesores con restricciones horarias: {len(profes_con_restr)}")

profes_con_restr[["id_profesor", col_restr]].head(10)


Número de profesores con restricciones horarias: 30


,id_profesor,restricciones
0,BIO1,1_2
3,BIO4,5_5
5,BIO6,4_5
6,ECO1,2_3
8,ECO3,4_3
10,ECO5,4_4
12,EF1,3_5
15,EF4,4_4
22,FR2,1_5
24,FR4,1_5


In [7]:
recuento_restricciones = (
    profes_con_restr[col_restr]
    .value_counts()
    .rename("num_profesores")
)

recuento_restricciones


restricciones
1_5    4
3_5    4
2_6    3
1_2    2
4_5    2
5_1    2
3_3    2
4_4    2
4_3    1
1_6    1
5_5    1
2_3    1
2_2    1
3_2    1
5_3    1
2_5    1
5_4    1
Name: num_profesores, dtype: int64

##AHORA VAMOS A PASAR A LA ESTRUCTURA DEL MODELO##



Conjunto de profesores

In [8]:
PROFESORES = df["id_profesor"].unique().tolist()
N_PROF = len(PROFESORES)

print(f"Número de profesores: {N_PROF}")


Número de profesores: 110


Conjunto de grupos

In [9]:
# columnas que representan grupos (ej: 1E_A, 2E_C, 1BACH_F, etc.)
# columnas binarias puras (solo 0 y 1)
COLUMNAS_BINARIAS = [
    c for c in df.columns
    if df[c].dropna().isin([0, 1]).all()
]

print("Columnas binarias:", COLUMNAS_BINARIAS)
print("Número:", len(COLUMNAS_BINARIAS))


GRUPOS = COLUMNAS_BINARIAS



Columnas binarias: ['1BACH_A', '1BACH_B', '1BACH_C', '1BACH_D', '1BACH_E', '1BACH_F', '1E_A', '1E_B', '1E_C', '1E_D', '1E_E', '1E_F', '1E_G', '2BACH_A', '2BACH_B', '2BACH_C', '2BACH_D', '2BACH_E', '2BACH_F', '2E_A', '2E_B', '2E_C', '2E_D', '2E_E', '2E_F', '2E_G', '3E_A', '3E_B', '3E_C', '3E_D', '3E_E', '3E_F', '4E_A', '4E_B', '4E_C', '4E_D', '4E_E', '4E_F']
Número: 38


Pasamos a definir los conjuntos de Días y Horas

In [10]:
DIAS = [1, 2, 3, 4, 5]          # lunes a viernes
HORAS = [1, 2, 3, 4, 5, 6]     # 6 horas lectivas

N_DIAS = len(DIAS)
N_HORAS = len(HORAS)


Ahora vamos a hacer un diccionario de compatibilidad profesor grupo

In [11]:
compatibilidad = {
    p: {
        g: int(df.loc[df["id_profesor"] == p, g].iloc[0])
        for g in GRUPOS
    }
    for p in PROFESORES
}

model = cp_model.CpModel()

x = {}
for p in PROFESORES:
    for g in GRUPOS:
        if compatibilidad[p][g] == 1:
            for d in DIAS:
                for h in HORAS:
                    x[p, g, d, h] = model.NewBoolVar(
                        f"x_{p}_{g}_{d}_{h}"
                    )




Vamos a pasar a poner las restricciones duras

Un profesor no puede dar dos clases a la vez y todos los grupos tienen 1 hora de clase en cada hora 

In [12]:
# Un profesor no puede estar en dos grupos a la vez

for p in PROFESORES:
    for d in DIAS:
        for h in HORAS:
            model.Add(
                sum(
                    x[p, g, d, h]
                    for g in GRUPOS
                    if (p, g, d, h) in x
                ) <= 1
            )


Un grupo no puede tener dos profesores a la vez 

In [13]:
# Un grupo no puede tener dos profesores a la vez

for g in GRUPOS:
    for d in DIAS:
        for h in HORAS:
            model.Add(
                sum(
                    x[p, g, d, h]
                    for p in PROFESORES
                    if (p, g, d, h) in x
                ) <= 1
            )


Ahora vamos a modelizar las horas semanales de cada asignatura 

In [14]:
import pandas as pd
from collections import defaultdict

df_pga = pd.read_csv("data/prueba_profesor_grupo_asignatura.csv")

asignatura_profesor_grupo = defaultdict(set)

for _, r in df_pga.iterrows():
    asignatura_profesor_grupo[(r["profesor"], r["grupo"])].add(r["asignatura"])

df_horas = pd.read_csv("data/prueba_horas_semanales.csv")

HorasSemanales = {
    (row.curso, row.asignatura): int(row.horas_semanales)
    for _, row in df_horas.iterrows()
}


# Asignaturas base (las que existen como materias "reales")
ASIGNATURAS_BASE = set(df_horas["asignatura"])

# Reglas de descomposición conocidas
MAPA_COMPUESTAS = {
    "RELORIENT": ("REL", "ORIENT"),
    "FRAUD": ("FR", "AUD"),
    "MUSFR": ("MUS", "FR"),
    "FILMUS": ("FIL", "MUS"),
}

# Quedarse solo con las que realmente existen en los datos
ASIGNATURAS_COMPUESTAS = {
    a: MAPA_COMPUESTAS[a]
    for a in ASIGNATURAS_BASE
    if a in MAPA_COMPUESTAS
}



def asignaturas_del_grupo(g):
    return set(
        df_pga.loc[df_pga["grupo"] == g, "asignatura"].unique()
    )




ASIGNATURAS_COMUNES = {}

for curso in df_pga["curso"].unique():
    grupos = df_pga.loc[df_pga["curso"] == curso, "grupo"].unique()

    conjuntos = [
        asignaturas_del_grupo(g)
        for g in grupos
    ]

    if conjuntos:
        ASIGNATURAS_COMUNES[curso] = set.intersection(*conjuntos)


ASIGNATURAS_MODALIDAD = {}

for _, row in df_pga[["curso", "letra"]].drop_duplicates().iterrows():
    curso = row["curso"]
    letra = row["letra"]
    g = f"{curso}_{letra}"

    propias = asignaturas_del_grupo(g) - ASIGNATURAS_COMUNES.get(curso, set())

    if propias:
        ASIGNATURAS_MODALIDAD[(curso, letra)] = propias



def tipo_grupo(g):
    curso, letra = g.split("_")

    # ESO sin modalidades
    if curso in {"1E", "2E", "3E"}:
        return curso

    # 4º ESO
    if curso == "4E":
        if letra in {"A", "C", "E", "F"}:
            return "4E_CIENCIAS"
        else:
            return "4E_SOCIALES"

    # 1º BACH
    if curso == "1BACH":
        if letra in {"A", "B", "C"}:
            return "1BACH_CIENCIAS"
        elif letra in {"D", "E"}:
            return "1BACH_SOCIALES"
        else:
            return "1BACH_HUMANIDADES"

    # 2º BACH
    if curso == "2BACH":
        if letra in {"A", "B", "C"}:
            return "2BACH_CIENCIAS"
        elif letra in {"D", "E"}:
            return "2BACH_SOCIALES"
        else:
            return "2BACH_HUMANIDADES"

    raise ValueError(f"Grupo no reconocido: {g}")

def curso_de_grupo(g):
    return g.split("_")[0]

def horas_semanales_grupo(g):
    curso = curso_de_grupo(g)
    asignaturas = asignaturas_del_grupo(g)

    horas = {}

    for a in asignaturas:
        if (curso, a) in HorasSemanales:
            horas[a] = HorasSemanales[(curso, a)]

    return horas


for g in GRUPOS:
    for asignatura in asignaturas_del_grupo(g):
        existe = any(
            asignatura in asignatura_profesor_grupo.get((p, g), set())
            for p in PROFESORES
        )
        if not existe:
            print("❌ Asignatura sin profesor:", g, asignatura)







In [15]:
# =========================================================
# Restricción: horas semanales por grupo y asignatura
# =========================================================

for g in GRUPOS:
    horas_dict = horas_semanales_grupo(g)

    for asignatura, horas in horas_dict.items():

        expr = [
            x[p, g, d, h]
            for p in PROFESORES
            for d in DIAS
            for h in HORAS
            if (p, g, d, h) in x
            and (p, g) in asignatura_profesor_grupo
            and asignatura in asignatura_profesor_grupo[(p, g)]
        ]

        model.Add(sum(expr) == horas)









Ahora hacemos que cuando se de una asignatura compuesta se ocupe una hora de cada profesor correspondiente

In [16]:
# =========================================================
# Restricción: asignaturas compuestas se imparten a la vez
# =========================================================


for g in GRUPOS:
    curso = curso_de_grupo(g)
    asignaturas_g = asignaturas_del_grupo(g)

    for compuesta, (a1, a2) in ASIGNATURAS_COMPUESTAS.items():

        # Solo si el grupo cursa la asignatura compuesta
        if compuesta not in asignaturas_g:
            continue

        for d in DIAS:
            for h in HORAS:

                expr_a1 = [
                    x[p, g, d, h]
                    for p in PROFESORES
                    if (p, g, d, h) in x
                    and (p, g) in asignatura_profesor_grupo
                    and a1 in asignatura_profesor_grupo[(p, g)]
                ]

                expr_a2 = [
                    x[p, g, d, h]
                    for p in PROFESORES
                    if (p, g, d, h) in x
                    and (p, g) in asignatura_profesor_grupo
                    and a2 in asignatura_profesor_grupo[(p, g)]
                ]

                if expr_a1 and expr_a2:
                    model.Add(sum(expr_a1) == sum(expr_a2))



No puede haber más de 4 grupos dando Tecnología

In [17]:
# RESTRICCIÓN: Máximo 4 grupos dando Tecnología simultáneamente

for d in DIAS:
    for h in HORAS:
        model.Add(
            sum(
                x[p, g, d, h]
                for g in GRUPOS
                for p in PROFESORES
                if (p, g, d, h) in x
                and (p, g) in asignatura_profesor_grupo
                and "TEC" in asignatura_profesor_grupo[(p, g)]
            ) <= 4
        )

No puede haber más de 3 grupos a la vez dando Música

In [18]:
ASIGNATURAS_USAN_MUSICA = {"MUS", "FILMUS", "MUSFR"}

# RESTRICCIÓN: Máximo 3 grupos usando aula de Música simultáneamente

for d in DIAS:
    for h in HORAS:
        model.Add(
            sum(
                x[p, g, d, h]
                for g in GRUPOS
                for p in PROFESORES
                if (p, g, d, h) in x
                and (p, g) in asignatura_profesor_grupo
                and asignatura_profesor_grupo[(p, g)].intersection(ASIGNATURAS_USAN_MUSICA)
            ) <= 3
        )

No puede haber más de 3 grupos dando EF

In [19]:
PROFESORES_EF = [p for p in PROFESORES if p.startswith("EF")]

for d in DIAS:
    for h in HORAS:
        model.Add(
            sum(
                x[p, g, d, h]
                for p in PROFESORES_EF
                for g in GRUPOS
                if (p, g, d, h) in x
            ) <= 3
        )

Un profesor debe dar al día 2 horas mínimo

In [20]:

# Variable auxiliar: y[p,d] = 1 si el profesor p trabaja el día d
y = {}

for p in PROFESORES:
    for d in DIAS:
        y[p, d] = model.NewBoolVar(f"trabaja_{p}_{d}")




for p in PROFESORES:
    for d in DIAS:
        model.Add(
            sum(
                x[p, g, d, h]
                for g in GRUPOS
                for h in HORAS
                if (p, g, d, h) in x
            ) >= 2 * y[p, d]
        )

M = len(GRUPOS) * len(HORAS)  # cota superior segura

for p in PROFESORES:
    for d in DIAS:
        model.Add(
            sum(
                x[p, g, d, h]
                for g in GRUPOS
                for h in HORAS
                if (p, g, d, h) in x
            ) <= M * y[p, d]
        )





Un profesor no puede dar más de 5 horas al día

In [21]:

# Máximo 5 horas lectivas al día por profesor

for p in PROFESORES:
    for d in DIAS:
        model.Add(
            sum(
                x[p, g, d, h]
                for g in GRUPOS
                for h in HORAS
                if (p, g, d, h) in x
            ) <= 5
        )
        

Restricciones particulares de profesores

In [22]:

# Restricciones horarias del profesorado

for p in PROFESORES:
    restr = df.loc[df["id_profesor"] == p, "restricciones"].iloc[0]

    if pd.notna(restr):
        d, h = map(int, restr.split("_"))

        model.Add(
            sum(
                x[p, g, d, h]
                for g in GRUPOS
                if (p, g, d, h) in x
            ) == 0
        )


Función que nos dice si un profesor da a un grupo en concreto que nos sirve para volver a ver si el csv es correcto

In [23]:
def profesor_da_a_grupo(p, g):
    """
    Devuelve True si el profesor p da alguna asignatura al grupo g
    """
    return (p, g) in asignatura_profesor_grupo


profesor_da_a_grupo("MAT1", "2BACH_A")  # True
        

True

Ahora pasamos a las restricciones blandas que son aquellas que solo penalizarán, si no se cumplen no pasa nada, el modelo no saldrá infactible

EF mejor a última hora

In [24]:

# -----------------------------------------
# BLANDA 1: EF en primeras horas (penaliza)
# -----------------------------------------

pen_ef = []

HORAS_MALAS_EF = [1, 2]   # primeras horas del día

PROFESORES_EF = [p for p in PROFESORES if p.startswith("EF")]

for p in PROFESORES_EF:
    for g in GRUPOS:
        for d in DIAS:
            for h in HORAS_MALAS_EF:
                if (p, g, d, h) in x:
                    v = model.NewBoolVar(f"pen_ef_{p}_{g}_{d}_{h}")
                    # si x=1 ⇒ v=1 (penaliza)
                    model.Add(x[p, g, d, h] <= v)
                    pen_ef.append(v)




Matemáticas, Inglés y Lengua mejor a primera hora

In [25]:

# ---------------------------------------------------
# BLANDA 2: Asignaturas core en horas tardías
# ---------------------------------------------------

pen_core = []

ASIGNATURAS_CORE = {"MAT", "LEN", "ING"}
HORAS_MALAS_CORE = [5, 6]   # últimas horas

for (p, g, d, h), var in x.items():
    if h in HORAS_MALAS_CORE:
        if (p, g) in asignatura_profesor_grupo:
            if any(a in ASIGNATURAS_CORE for a in asignatura_profesor_grupo[(p, g)]):
                v = model.NewBoolVar(f"pen_core_{p}_{g}_{d}_{h}")
                model.Add(var <= v)
                pen_core.append(v)



Penalizamos que matemáticas, inglés o lengua vayan seguidas de una de las anteriores

In [26]:
pen_densas = []
for g in GRUPOS:
    for d in DIAS:
        for h in HORAS:
            if h == max(HORAS):
                continue  # no existe h+1

            # variables que indican "hora h es densa"
            expr_h = [
                x[p, g, d, h]
                for p in PROFESORES
                if (p, g, d, h) in x
                and (p, g) in asignatura_profesor_grupo
                and any(
                    a in ASIGNATURAS_CORE
                    for a in asignatura_profesor_grupo[(p, g)]
                )
            ]

            # variables que indican "hora h+1 es densa"
            expr_h1 = [
                x[p, g, d, h + 1]
                for p in PROFESORES
                if (p, g, d, h + 1) in x
                and (p, g) in asignatura_profesor_grupo
                and any(
                    a in ASIGNATURAS_CORE
                    for a in asignatura_profesor_grupo[(p, g)]
                )
            ]

            if not expr_h or not expr_h1:
                continue

            # variable de penalización
            pen = model.NewBoolVar(f"pen_densa_{g}_{d}_{h}")

            # pen = 1 si ambas horas son densas
            model.Add(sum(expr_h) + sum(expr_h1) <= 1 + pen)
            model.Add(pen <= sum(expr_h))
            model.Add(pen <= sum(expr_h1))

            pen_densas.append(pen)


Intentar que los profesores no tengan huecos en su horarios

In [27]:


# ---------------------------------------------------
# BLANDA 4: Penalizar huecos en horario del profesor
# ---------------------------------------------------

pen_huecos = []

for p in PROFESORES:
    for d in DIAS:
        for h in range(min(HORAS), max(HORAS) - 1):

            # variables auxiliares: trabaja o no
            t_h  = model.NewBoolVar(f"t_{p}_{d}_{h}")
            t_h1 = model.NewBoolVar(f"t_{p}_{d}_{h+1}")
            t_h2 = model.NewBoolVar(f"t_{p}_{d}_{h+2}")

            model.Add(
                sum(x[p, g, d, h] for g in GRUPOS if (p, g, d, h) in x) >= t_h
            )
            model.Add(
                sum(x[p, g, d, h+1] for g in GRUPOS if (p, g, d, h+1) in x) >= t_h1
            )
            model.Add(
                sum(x[p, g, d, h+2] for g in GRUPOS if (p, g, d, h+2) in x) >= t_h2
            )

            gap = model.NewBoolVar(f"gap_{p}_{d}_{h}")

            # gap = 1 si hay hueco
            model.Add(t_h + t_h2 - t_h1 <= gap)

            pen_huecos.append(gap)



Otra restricción blanda que nos va penalizar que se de alguna asignatura fuerte a última hora

In [28]:
ASIGNATURAS_DENSAS = {
    "MAT", "LEN", "ING",
    "FYQ", "BIO",
    "FISICA", "QUIMICA"
}


ULTIMA_HORA = max(HORAS)
pen_densa_ultima = []

for g in GRUPOS:
    asignaturas_g = asignaturas_del_grupo(g)

    for a in asignaturas_g:
        if a not in ASIGNATURAS_DENSAS:
            continue

        for d in DIAS:
            for p in PROFESORES:

                if (
                    (p, g) in asignatura_profesor_grupo
                    and a in asignatura_profesor_grupo[(p, g)]
                    and (p, g, d, ULTIMA_HORA) in x
                ):
                    pen = model.NewBoolVar(
                        f"pen_densa_ultima_{a}_{g}_{d}_{p}"
                    )

                    # Si hay clase → penaliza
                    model.Add(
                        pen >= x[p, g, d, ULTIMA_HORA]
                    )

                    pen_densa_ultima.append(pen)


Ahora vamos a pasar a poner la función objetivo junto con los valores de las penalizaciones y su normalización por que tal y como dijo mi profesor un día no podemos mezclar el tocino con la velocidad

In [29]:
MAX_EF     = max(1, len(pen_ef))
MAX_CORE   = max(1, len(pen_core))
MAX_HUECOS = max(1, len(pen_huecos))
MAX_DENSAS = max(1, len(pen_densas))
MAX_DENSA_ULTIMA = max(1, len(pen_densa_ultima))


W_DENSAS = 4 / MAX_DENSAS
W_EF     = 5 / MAX_EF
W_CORE   = 6 / MAX_CORE
W_HUECOS = 8 / MAX_HUECOS
W_DENSA_ULTIMA = 5 / MAX_DENSA_ULTIMA

model.Minimize(
    W_EF     * sum(pen_ef)
  + W_CORE   * sum(pen_core)
  + W_HUECOS * sum(pen_huecos)
  + W_DENSAS * sum(pen_densas)
  + W_DENSA_ULTIMA * sum(pen_densa_ultima)
)



solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 20*60
solver.parameters.num_search_workers = 8     # usa todos los núcleos


status = solver.Solve(model)

print(solver.StatusName(status))

print("Valor objetivo:", solver.ObjectiveValue())




OPTIMAL
Valor objetivo: 0.4751315789473692


Importamos el horario a un csv

In [30]:
import os
import csv



output_dir = "horarios_cursos"
os.makedirs(output_dir, exist_ok=True)

for g in GRUPOS:

    filename = os.path.join(output_dir, f"{g}.csv")

    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["Grupo", "Dia", "Hora", "Profesor", "Asignatura"])

        for d in DIAS:
            for h in HORAS:
                for p in PROFESORES:
                    if (p, g, d, h) in x and solver.Value(x[p, g, d, h]) == 1:

                        # obtener asignatura que da ese profesor a ese grupo
                        asignaturas = asignatura_profesor_grupo.get((p, g), [])
                        asignatura = next(iter(asignaturas)) if asignaturas else ""

                        writer.writerow([g, d, h, p, asignatura])


for g in GRUPOS:
    path = f"horarios_cursos/{g}.csv"
    df = pd.read_csv(path)
    df = df.sort_values(["Dia", "Hora"])
    df.to_csv(path, index=False)




